# MCS precipitation statistics


In [8]:
import numpy as np
import os
import glob 
import random 

import matplotlib.pyplot as plt
import cartopy
from mpl_toolkits.basemap import Basemap
import matplotlib.colors as colors

import scipy
from scipy import ndimage
from scipy.stats import skew
from scipy.ndimage import label, generate_binary_structure
import matplotlib.pyplot as plt

from netCDF4 import Dataset
from collections import Counter 

import pandas as pd 

In [65]:
# read in statistics for one month 
file = '/media/juli/Elements/GPM_IMERG_F_v05/GPM_finalrun/tracks/201708_tracks.nc'
def read_in_netcdf(file):
    ds = xr.open_dataset(file)
    stats = ds.to_dataframe()
    return stats

In [66]:
stats = read_in_netcdf(file)

In [67]:
# add new dataframe to current one 
file2= '/media/juli/Elements/GPM_IMERG_F_v05/GPM_finalrun/tracks/201608_tracks.nc'
df= read_in_netcdf(file2)
stats.append(df)


date  time        lon         lat  PREC_mean   PREC_max  PREC_min  \
ID                                                                           
7    20170801  1230  30.144594   75.495956   9.975197  15.294099  7.086372   
7    20170801  2230  32.579533   74.223152  15.149220  38.801167  7.011247   
7    20170802  1430  27.377781   87.183327  12.404141  29.119394  7.854423   
7    20170802  1500  27.832500   92.294174  13.751048  54.508476  7.050000   
7    20170802  1530  27.350000   87.263107  12.730351  37.800003  7.050000   
7    20170802  1600  27.391178   87.197052  12.357388  30.532213  7.002930   
7    20170802  1630  27.306177   87.498314  15.499947  42.808884  7.028215   
7    20170802  1700  27.395237   86.819046  13.641740  33.191288  7.130544   
12   20170803  1500  28.317188   81.545319  12.360316  24.347271  7.068916   
12   20170803  1530  28.197775   81.726669  14.875180  42.217518  7.042878   
12   20170803  1600  28.218603   81.712791  14.314378  37.151413  7.042878   
12   20170803  1630  28.516155   81.191536  11.253471  24.897930  7.042878   
12   20170803  1700  28.283026   81.795868  11.185827  23.103165  7.000862   
12   20170803  1730  28.240322   81.906776  10.831292  36.353161  7.011166   
12   20170803  1800  28.852564   80.648460   9.616864  20.014809  7.002182   
12   20170803  1830  28.829227   80.706520  10.153513  30.303595  7.015959   
12   20170803  1900  28.682632   80.841049  10.855099  41.555943  7.001553   
12   20170803  2000  28.020502   82.228416  13.066608  37.389618  7.107280   
12   20170803  2030  28.156916   82.149467  12.244668  29.565128  7.001795   
12   20170803  2100  29.174854   80.499428  12.418964  30.784742  7.008637   
12   20170803  2130  29.138035   80.763504  16.632452  60.121552  7.003118   
12   20170803  2200  28.319378   82.263870  15.424510  49.136753  7.090671   
12   20170803  2230  28.305954   82.183632  11.155113  24.256348  7.024961   
12   20170803  2300  28.364008   82.100197  10.543510  19.671795  7.013684   
12   20170803  2330  28.575138   81.803627   9.553509  17.239912  7.067589   
12   20170804     0  29.112207   80.852371  10.577977  23.393875  7.007197   
12   20170804   100  28.965248   81.069504  16.760117  57.215076  7.016952   
12   20170804   130  28.843451   81.318130  13.721951  37.364948  7.016952   
21   20170804  1600  32.145123   75.318291  16.661455  36.705730  7.443247   
21   20170804  1830  27.531080   85.722977  16.601643  29.852373  7.307247   
..        ...   ...        ...         ...        ...        ...       ...   
219  20160822  1100  34.956451  101.979027  14.482363  34.501595  7.010418   
219  20160822  1130  34.893013  101.791939  13.423845  40.588009  7.014668   
219  20160822  1200  34.715454  102.291817  12.328421  53.790771  7.040586   
219  20160822  1230  34.746666  102.302223  11.721567  25.248728  7.014534   
219  20160822  1300  34.745602  102.292862  11.464256  20.873192  7.069605   
237  20160825   330  31.610868   74.243477  11.600687  21.697721  7.006582   
237  20160825   400  31.565214   74.093483  14.845578  34.792549  7.065018   
237  20160825   430  31.610918   74.162651  13.143086  24.583767  7.131778   
237  20160825   500  31.671783   74.178719  13.340077  28.150284  7.099186   
237  20160825   530  31.660841   74.213860  12.646046  21.644279  7.011168   
237  20160825   600  31.731482   74.251862  10.467327  18.598202  7.098316   
237  20160825   700  31.531691   74.057053  14.016959  24.930681  7.043148   
237  20160825   730  31.511971   73.955643  17.451811  44.801952  7.099186   
237  20160825   800  31.540627   73.976562  13.856440  27.333822  7.078027   
237  20160825   830  31.557020   74.011414  13.218724  25.478609  7.265245   
237  20160825   900  31.748180   74.160912  12.445734  23.805929  7.385150   
247  20160827   700  31.244774   74.586563  13.936036  26.376822  7.019487   
247  20160827   730  31.247686   74.443642  19.117899  69.715134  7.089238   
247  20160827   800  31.171793 

In [46]:
# average get average values for entire MCS subset
mean_values= stats.mean(axis= 0)
mean_values

date         2.017081e+07
time         1.400769e+03
lon          2.946814e+01
lat          8.862515e+01
PREC_mean    1.338637e+01
PREC_max     3.435280e+01
PREC_min     7.078675e+00
area         1.461640e+04
skew         1.420062e+00
dtype: float64

In [64]:
# plot PDF of parameter in subset 

for i in stats.columns[4:6]:
    plt.figure(figsize= (7,5)  )
    plt.hist(stats[i], bins = int(180/1),color = 'black', edgecolor = 'black')
    plt.rcParams.update({'font.size': 25})
    plt.ylabel('Frequency ')
    plt.xlabel(i)
    plt.show()


In [68]:
# calculate lifetime of MCS 

In [69]:
# plot track of MCS 